In [1]:
macro bash_str(s) open(`bash`,"w",STDOUT) do io; print(io, s); end; end

@bash_str (macro with 1 method)

In [6]:
bash"""
cd ../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/
julia Solve.jl
"""

tx_inits = Int64[]
FB = [0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100.0; 0.0 100

In [5]:
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/Include.jl")
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/Bounds.jl")
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/TXTLDictionary.jl")

# load the data dictionary -
data_dictionary = DataDictionary(0,0,0)
TXTL_dictionary = TXTLDictionary(0,0,0,data_dictionary)

#Set objective reaction
data_dictionary["objective_coefficient_array"][find_idx("but_export", data_dictionary)] = -1;

#=============================Cases=========================================#
#Define case number
# 1 = Amino Acid Uptake & Synthesis
# 2 = Amino Acid Uptake w/o Synthesis
# 3 = Amino Acid Synthesis w/o Uptake
Case = 1
if Case == 1
  data_dictionary["AASyn"] = 100;
  data_dictionary["AAUptake"] = 30
  data_dictionary["AASecretion"] = 0;
end
if Case == 2
  data_dictionary["AASyn"] = 0;
  data_dictionary["AAUptake"] = 30
  data_dictionary["AASecretion"] = 0;
end
if Case == 3
  data_dictionary["AASyn"] = 100;
  data_dictionary["AAUptake"] = 0
  data_dictionary["AASecretion"] = 100;
end

#Set Promoter
#1 = T7 Promoter model
#2 = P70a Promoter model
Promoter_model = 1
#===========================================================================#

#Set Plasmid Dose (nM)
plasmid_concentration = 13.3;
volume = TXTL_dictionary["volume"]
gene_copy_number = (volume/1e9)*(6.02e23)*plasmid_concentration;
TXTL_dictionary["gene_copies"] = gene_copy_number

#Set Glucose and Oxygen (mM/h)
data_dictionary["GlcUptake"] = 30
data_dictionary["Oxygen"] = 100;

# solve the lp problem -
data_dictionary = Bounds(data_dictionary,TXTL_dictionary);
(objective_value, flux_array, dual_array, uptake_array, exit_flag) = FluxDriver(data_dictionary)
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/Performance.jl")

LoadError: [91mSystemError: opening file Network.dat: No such file or directory[39m

In [5]:
data_dictionary

Dict{AbstractString,Any} with 21 entries:
  "is_minimum_flag"                     => true
  "objective_coefficient_array"         => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
  "AASecretion"                         => 0
  "characteristic_enzyme_abundance_mM"  => 4.73269e-5
  "species_bounds_array"                => [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 …
  "mass_of_single_cell"                 => 2.8e-13
  "AASyn"                               => 100
  "volume_of_cell"                      => 6.96909e-16
  "list_of_metabolite_symbols"          => String["M_10fthf_c", "M_13dpg_c", "M…
  "txtl_parameter_dictionary"           => Dict{AbstractString,Any}(Pair{Abstra…
  "species_abundance_array"             => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, …
  "stoichiometric_matrix"               => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.…
  "number_of_reactions"                 => 282
  "default_flux_bounds_array"           => [0.0 22.149; 0.0 22.149; … ; 0.0 0.0…
  "Oxygen"                              => 1

In [25]:
data_dictionary["list_of_reaction_strings"]

178-element Array{String,1}:
 "R_glk_atp::M_atp_c+M_glc_D_c --> M_adp_c+M_g6p_c"                   
 "R_pgi::M_g6p_c --> M_f6p_c"                                         
 "R_pgi_reverse::M_f6p_c --> M_g6p_c"                                 
 "R_pfk::M_atp_c+M_f6p_c --> M_adp_c+M_fdp_c"                         
 "R_fdp::M_fdp_c+M_h2o_c --> M_f6p_c+M_pi_c"                          
 "R_fbaA::M_fdp_c --> M_dhap_c+M_g3p_c"                               
 "R_fbaA_reverse::M_dhap_c+M_g3p_c --> M_fdp_c"                       
 "R_tpiA::M_dhap_c --> M_g3p_c"                                       
 "R_tpiA_reverse::M_g3p_c --> M_dhap_c"                               
 "R_gapA::M_g3p_c+M_nad_c+M_pi_c --> M_13dpg_c+M_h_c+M_nadh_c"        
 "R_gapA_reverse::M_13dpg_c+M_h_c+M_nadh_c --> M_g3p_c+M_nad_c+M_pi_c"
 "R_pgk::M_13dpg_c+M_adp_c --> M_3pg_c+M_atp_c"                       
 "R_pgk_reverse::M_3pg_c+M_atp_c --> M_13dpg_c+M_adp_c"               
 ⋮                                              

In [138]:
outfile = open("../BTOL/Bounds.txt", "w")
writedlm(outfile,data_dictionary["default_flux_bounds_array"]);
close(outfile);